In [1]:
import os
import json
import numpy as np
from omegaconf import OmegaConf

In [2]:
exp_dir = "../../../logs/en/asr/evaluate"
folders = {
    "Fine-tuned_9": "version_9",
    "AC_1": "version_48",
    "AC_2": "version_49",
    "AC_5": "version_50",
    "AC_10": "version_24",
}

In [3]:
# define which accent is standard, which are seen and which are unseen
classes = {
    "seen": ["us", "uk", "ca", "in", "de", "au"],
    "unseen": ["hk", "sg", "sc", "nz", "ie", "za", "ni", "ph"],
}

In [4]:
# load WERs of each file, for each accent
wers = dict()
for exp, folder in folders.items():
    wers[exp] = {k[5:-4]: v for k, v in json.load(open(f"{exp_dir}/{folder}/avg_wers.json")).items()}

In [5]:
# transform dictionary into a numpy array, where one axis represents the accents and another the experiments
experiments = list(wers.keys())
accents = list(wers[experiments[0]].keys())
n_words = np.array([wers[experiments[0]][acc]["n_words"] for acc in accents])
avg_wers = np.array([[wers[exp][acc]["avg_wer"] for exp in experiments] for acc in accents])

In [6]:
# compute means for each class, for each experiment
class_indices = {k: [accents.index(acc) for acc in v] for k, v in classes.items()}
class_means = {k: np.mean(avg_wers[indices], axis=0) for k, indices in class_indices.items()}

In [7]:
# print the avg. WERs and the means (overall and per class) as a markdown table
headers = ["Accent / Dialect"] + experiments
print((" | ").join(headers))
print(f"|{'---:|'*len(headers)}")

# print avg. WERs
for i in range(len(accents)):
    row = [accents[i]] + [f"{avg_wers[i,j]*100:.2f}" for j in range(len(experiments))]
    print((" | ").join(row))

# print overall means
row = ["mean"] + [f"{np.mean(avg_wers[:,i])*100:.2f}" for i in range(len(experiments))]
print((" | ").join(row))

# print class means
for key, value in class_means.items():
    row = [f"{key} mean"] + [f"{value[i]*100:.2f}" for i in range(len(experiments))]
    print((" | ").join(row))

# print worst avg. WER of each experiment
row = ["worst"] + [f"{v*100:.2f}" for v in np.max(avg_wers, axis=0)]
print((" | ").join(row))

# print the avg. of the three worst avg. WERs of each experiment
row = ["3-worst mean."] + [f"{np.mean(np.sort(avg_wers, axis=0)[-3:,i])*100:.2f}" for i in range(len(experiments))]
print((" | ").join(row))


Accent / Dialect | Fine-tuned_9 | AC_1 | AC_2 | AC_5 | AC_10
|---:|---:|---:|---:|---:|---:|
za | 6.54 | 6.51 | 6.52 | 6.58 | 6.54
sg | 10.41 | 11.07 | 11.09 | 10.47 | 10.51
in | 9.21 | 9.30 | 9.30 | 9.34 | 9.41
au | 7.47 | 7.53 | 7.52 | 7.61 | 7.53
hk | 9.76 | 10.22 | 10.21 | 9.87 | 9.75
ie | 7.51 | 7.64 | 7.66 | 7.45 | 7.41
uk | 5.78 | 5.80 | 5.80 | 5.90 | 5.87
ph | 8.90 | 9.08 | 9.11 | 8.81 | 8.79
de | 5.62 | 5.62 | 5.61 | 5.61 | 5.66
ni | 6.96 | 6.81 | 6.79 | 7.16 | 7.02
us | 5.43 | 5.67 | 5.65 | 5.46 | 5.46
ca | 5.36 | 5.51 | 5.49 | 5.37 | 5.35
nz | 5.40 | 5.77 | 5.76 | 5.38 | 5.28
sc | 33.50 | 33.80 | 33.94 | 33.15 | 32.55
mean | 9.13 | 9.31 | 9.32 | 9.15 | 9.08
seen mean | 6.48 | 6.57 | 6.56 | 6.55 | 6.55
unseen mean | 11.12 | 11.36 | 11.39 | 11.11 | 10.98
worst | 33.50 | 33.80 | 33.94 | 33.15 | 32.55
3-worst mean. | 17.89 | 18.36 | 18.41 | 17.83 | 17.61


Accent / Dialect | AC_1 | AC_2 | AC_5 | AC_10
|---:|---:|---:|---:|---:|
Southern African (South Africa, Zimbabwe, Namibia) | 6.51 | 6.52 | 6.58 | 6.54
Singaporean English | 11.07 | 11.09 | 10.47 | 10.51
India and South Asia (India, Pakistan, Sri Lanka) | 9.30 | 9.30 | 9.34 | 9.41
Australian English | 7.53 | 7.52 | 7.61 | 7.53
Hong Kong English | 10.22 | 10.21 | 9.87 | 9.75
Irish English | 7.64 | 7.66 | 7.45 | 7.41
England English | 5.80 | 5.80 | 5.90 | 5.87
Filipino | 9.08 | 9.11 | 8.81 | 8.79
German English,Non native speaker | 5.62 | 5.61 | 5.61 | 5.66
Northern Irish | 6.81 | 6.79 | 7.16 | 7.02
United States English | 5.67 | 5.65 | 5.46 | 5.46
Canadian English | 5.51 | 5.49 | 5.37 | 5.35
New Zealand English | 5.77 | 5.76 | 5.38 | 5.28
Scottish English | 33.80 | 33.94 | 33.15 | 32.55
mean | 9.31 | 9.32 | 9.15 | 9.08
seen mean | 6.57 | 6.56 | 6.55 | 6.55
unseen mean | 11.36 | 11.39 | 11.11 | 10.98
worst | 33.80 | 33.94 | 33.15 | 32.55
3-worst mean. | 18.36 | 18.41 | 17.83 | 17.61

In [8]:
# print the experiment folders of each experiment (both train and eval folders)
print("Experiment folders:\n")
for exp, folder in folders.items():
    eval_folder = os.path.join(exp_dir, folder)
    eval_config = OmegaConf.load(os.path.join(eval_folder, "config.yaml"))
    train_folder = f'../{eval_config.asr.ckpt.replace("/checkpoints/last.ckpt", "")}'
    print(f"- {exp}: training `{train_folder[3:]}`, evaluation `{eval_folder[3:]}`")

Experiment folders:

- Fine-tuned_9: training `logs/en/asr/train/version_11`, evaluation `../../logs/en/asr/evaluate/version_9`
- AC_1: training `logs/en/ensemble/train/binary/b7/DAT/version_90`, evaluation `../../logs/en/asr/evaluate/version_48`
- AC_2: training `logs/en/ensemble/train/binary/b7/DAT/version_91`, evaluation `../../logs/en/asr/evaluate/version_49`
- AC_5: training `logs/en/ensemble/train/binary/b7/DAT/version_92`, evaluation `../../logs/en/asr/evaluate/version_50`
- AC_10: training `logs/en/ensemble/train/binary/b7/DAT/version_66`, evaluation `../../logs/en/asr/evaluate/version_24`
